In [2]:
from bs4 import BeautifulSoup
import requests
import MeCab as mc
import json
import sqlite3
import requests
import records
import glob
import json
import urllib.parse
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import gensim
from gensim.models import Word2Vec
from gensim.corpora.dictionary import Dictionary
from gensim import corpora, matutils
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from joblib import Parallel,delayed

In [18]:
conn = None

def connect():
    global conn
    conn = sqlite3.connect('./restaurant.db')
    
def close():
    conn.close()
    
def create_table():
    conn.execute('DROP TABLE IF EXISTS RESTAURANTS')
    conn.execute('''CREATE TABLE RESTAURANTS(
    id               INTEGER PRIMARY KEY AUTOINCREMENT,
    restaurant_name            TEXT
    
    )''')
    conn.execute('DROP TABLE IF EXISTS REVIEWS')
    conn.execute('''CREATE TABLE REVIEWS(
    id               INTEGER PRIMARY KEY AUTOINCREMENT,
    restaurant_id              TEXT,
    review                     TEXT,
    rate                       FLOAT
    )''')
    

    
def load(restaurant_name):
    conn.execute(
    'INSERT INTO RESTAURANTS (restaurant_name) VALUES (?)', (restaurant_name,))
    conn.commit()

def load_1(restaurant_id ,review ,rate):
    conn.execute(
    'INSERT INTO REVIEWS (restaurant_id,review ,rate) VALUES (?,?,?)', (restaurant_id ,review ,rate,))
    conn.commit()

In [19]:
connect()
create_table()


In [21]:
url='https://tabelog.com/rstLst/hamburger/?SrtT=rvcn&sk=%E3%83%8F%E3%83%B3%E3%83%90%E3%83%BC%E3%82%AC%E3%83%BC&svd=20190530&svt=1900&svps=2&Srt=D'
r = requests.get(url)
bs = BeautifulSoup(r.content,'html.parser')
table = bs.find('ul', class_='js-rstlist-info rstlist-info')
# print(table)

name = []
for a in table.find_all('a', class_='list-rst__rst-name-target cpy-rst-name'):
    name.append(a.text) 

name_new = name[:10]
len(name_new)

10

In [22]:
for restaurant_name in name_new:
    load(restaurant_name)

In [7]:
# 进入评论数子的那个网页里面，把每一个店的评论页面的link找出来
comment_link=[]
for a in table.find_all('a', class_='list-rst__rvw-count-target'):
    i=0
    comment_link.append(a.get('href'))
comment_link_new = comment_link[:10]
# comment_link_new

In [8]:
# 找到了评论区的链接，进入店面抽取单个评论的链接
comments_link=[]
comment_number=[]
header_r = 'https://tabelog.com/'
for one_link in comment_link_new[:10]:
    i = 1
    number = 1
    while True:
        com_link = one_link+'COND-0/smp1/?smp=1&lc=0&rvw_part=all&PG='+str(i)
        com_links = requests.get(com_link)
        com_linksbs = BeautifulSoup(com_links.content,'html.parser')
        if com_linksbs.find('a', class_='c-pagination__arrow c-pagination__arrow--next'):
            events=com_linksbs.find_all('div', class_='rvw-item js-rvw-item-clickable-area')
            for event in events:
                comments_link.append([number,event.get('data-detail-url')])
            i+=1
        else:
            com_link = one_link+'COND-0/smp1/?smp=1&lc=0&rvw_part=all&PG='+str(i)
            com_links = requests.get(com_link)
            com_linksbs = BeautifulSoup(com_links.content,'html.parser')
            events=com_linksbs.find_all('div', class_='rvw-item js-rvw-item-clickable-area')
            for event in events:
                comments_link.append([number,event.get('data-detail-url')])
            break
    number+=1


# print(len(comments_link))   
 
#  找到的链接beautifulsoup一下再进行单个评论的链接抽取 
   
  
   

In [39]:
# link_a = []
# comment = []
# rate = []
# header_r = 'https://tabelog.com/'
# for a in comments_link:
#     r = header_r + a
#     link_a.append(r)
# for every in link_a:
#     every_r = requests.get(every)
#     every_bs = BeautifulSoup(every_r.content,'html.parser')
#     rate_number=every_bs.find('b', class_='c-rating__val c-rating__val--strong').text
#     rate.append(rate_number)
#     com=every_bs.find('div', class_='rvw-item__rvw-comment').find('p').text
#     comment.append(com)

TypeError: must be str, not list

In [9]:
def proc(id_, u):    
    header_r = 'https://tabelog.com/'
    com_html = requests.get(header_r + u).text
    com_html_bs = BeautifulSoup(com_html, "html.parser")
    rates = com_html_bs.find('b', class_='c-rating__val c-rating__val--strong').text
    rate = float(rates) if rates != "-" else 0.0
    review = com_html_bs.find("div", class_="rvw-item__rvw-comment").find('p').text
    print(u)
    return[id_,rate,review]

In [26]:
proc(1,'tokyo/A1302/A130204/13003002/dtlrvwlst/B415485881/?use_type=0&smp=1')

tokyo/A1302/A130204/13003002/dtlrvwlst/B415485881/?use_type=0&smp=1


[1,
 3.7,
 '\n          日曜日の17時過ぎに伺いました＼(^o^)／駅から徒歩4〜5分の所の交差点の角にあります(>_<)外観は赤がメインですが、落ち着いた赤色⁉︎なので下品さは無いです(>_<)店内はカウンター席とテーブル席がありますが店内も赤い(>_<)でも、スカーレット⁉︎ワインレッド⁉︎なのでお洒落な感じで素敵な店内です＼(^o^)／⚪︎LOT BURGER（ロットバーガー）1650円※ソースはBBQ、テリヤキ、レッドホットチリ、スイートチリから選べるらしく1番人気のBBQをチョイス＼(^o^)／ベーコン、エッグ、チーズパイナップルがトッピングされてると聞いた時…一瞬、パイナップルかぁ〜.(￣◇￣;)と思いましたが、食べてビックリ美味しいしパイナップルが邪魔してない(>_<)（笑）というか、時々、影を見せてくるパイナップルが美味しい♪( ´θ｀)ノしっかりと合ってる(>_<)そして、レタスがシャキシャキで美味しい＼(^o^)／もちろん、パテもバンズも美味しいしチーズも最高＼(^o^)／でも、凄いのは全ての食材のバランスというか何か１つが凄いんじゃなく、みんながすごい（笑）凄く美味しいバーガーでした♪( ´θ｀)ノ\n        ']

In [13]:
comments_link[0]

[1,
 '/tokyo/A1302/A130204/13003002/dtlrvwlst/B415485881/?use_type=0&srt=&sby=&rvw_part=all&lc=0&smp=1']

In [18]:
# header_r +  comments_link[0]
# print(link_a)

['https://tabelog.com//tokyo/A1302/A130204/13003002/dtlrvwlst/B415485881/?use_type=0&srt=&sby=&rvw_part=all&lc=0&smp=1', 'https://tabelog.com//tokyo/A1302/A130204/13003002/dtlrvwlst/B415224498/?use_type=0&srt=&sby=&rvw_part=all&lc=0&smp=1', 'https://tabelog.com//tokyo/A1302/A130204/13003002/dtlrvwlst/B415032944/?use_type=0&srt=&sby=&rvw_part=all&lc=0&smp=1', 'https://tabelog.com//tokyo/A1302/A130204/13003002/dtlrvwlst/B83601171/?use_type=0&srt=&sby=&rvw_part=all&lc=0&smp=1', 'https://tabelog.com//tokyo/A1302/A130204/13003002/dtlrvwlst/B4070603/?use_type=0&srt=&sby=&rvw_part=all&lc=0&smp=1', 'https://tabelog.com//tokyo/A1302/A130204/13003002/dtlrvwlst/B3016674/?use_type=0&srt=&sby=&rvw_part=all&lc=0&smp=1', 'https://tabelog.com//tokyo/A1302/A130204/13003002/dtlrvwlst/B412900681/?use_type=0&srt=&sby=&rvw_part=all&lc=0&smp=1', 'https://tabelog.com//tokyo/A1302/A130204/13003002/dtlrvwlst/B392617736/?use_type=0&srt=&sby=&rvw_part=all&lc=0&smp=1', 'https://tabelog.com//tokyo/A1302/A130204/13

In [17]:
# every_r = requests.get(header_r +  comments_link[0])
# every_bs = BeautifulSoup(every_r.content,'html.parser')
# rate_number=every_bs.find('b', class_='c-rating__val c-rating__val--strong').text
# print(rate_number)
# com=every_bs.find('div', class_='rvw-item__rvw-comment').find('p').text
# print(com)

3.7

          日曜日の17時過ぎに伺いました＼(^o^)／駅から徒歩4〜5分の所の交差点の角にあります(>_<)外観は赤がメインですが、落ち着いた赤色⁉︎なので下品さは無いです(>_<)店内はカウンター席とテーブル席がありますが店内も赤い(>_<)でも、スカーレット⁉︎ワインレッド⁉︎なのでお洒落な感じで素敵な店内です＼(^o^)／⚪︎LOT BURGER（ロットバーガー）1650円※ソースはBBQ、テリヤキ、レッドホットチリ、スイートチリから選べるらしく1番人気のBBQをチョイス＼(^o^)／ベーコン、エッグ、チーズパイナップルがトッピングされてると聞いた時…一瞬、パイナップルかぁ〜.(￣◇￣;)と思いましたが、食べてビックリ美味しいしパイナップルが邪魔してない(>_<)（笑）というか、時々、影を見せてくるパイナップルが美味しい♪( ´θ｀)ノしっかりと合ってる(>_<)そして、レタスがシャキシャキで美味しい＼(^o^)／もちろん、パテもバンズも美味しいしチーズも最高＼(^o^)／でも、凄いのは全ての食材のバランスというか何か１つが凄いんじゃなく、みんながすごい（笑）凄く美味しいバーガーでした♪( ´θ｀)ノ
        


In [10]:
r = Parallel(n_jobs=-1)([delayed(proc)(id_,u) for id_,u in comments_link])
len(r)

6105

In [20]:
for data in r:
    load_1(data[0],data[2],data[1])

In [23]:
close()